# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [1]:
with open("real_estate_sales_data.txt",encoding="utf-8") as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([real_estate_sales])

In [5]:
docs[0]

Document(page_content='[客户问题] 你们的手机相机效果怎么样？  \n[销售回答] 华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。')

In [6]:
len(docs)

37

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(base_url="https://burn.hair/v1"))

c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [8]:
query = "手机贵不贵"

In [9]:
answer_list = db.similarity_search(query)

In [10]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 这款手机的价格是否划算？  
[销售回答] 华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。

[客户问题] 这款手机的颜值怎么样？  
[销售回答] 华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。

[客户问题] 这款手机适合玩游戏吗？  
[销售回答] 这款华为手机性能非常强大，搭载了高性能处理器和GPU，绝对能够满足您玩大型游戏的需求，同时我们还有游戏模式，进一步提升游戏体验。

[客户问题] 华为的手机和其他品牌相比有什么优势？  
[销售回答] 华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。


In [11]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [12]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [13]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000024D6BEAEA10>, search_kwargs={'k': 3})

In [14]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 这款手机的价格是否划算？  
[销售回答] 华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。

[客户问题] 这款手机的颜值怎么样？  
[销售回答] 华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。

[客户问题] 这款手机适合玩游戏吗？  
[销售回答] 这款华为手机性能非常强大，搭载了高性能处理器和GPU，绝对能够满足您玩大型游戏的需求，同时我们还有游戏模式，进一步提升游戏体验。


In [15]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的手机啊？")

In [16]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款手机的价格是否划算？  
[销售回答] 华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。

[客户问题] 这款手机有红外功能吗？  
[销售回答] 是的，这款华为手机配备了红外功能，您可以用手机遥控家电，非常实用。

[客户问题] 这款手机的颜值怎么样？  
[销售回答] 华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。


#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [17]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [18]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 这款手机的价格是否划算？  
[销售回答] 华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。


### 提取向量数据库中的`销售回答`

In [19]:
docs = retriever.get_relevant_documents(query)

In [20]:
docs[0].page_content

'[客户问题] 这款手机的价格是否划算？  \n[销售回答] 华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。'

In [21]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 这款手机的价格是否划算？  \n',
 '华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。']

In [22]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [23]:
ans

'华为手机提供了高性价比的选择，您不仅能享受顶尖的技术和优质的售后服务，还能获得持续的软件更新和优化，绝对物超所值。'

#### 尝试各种问题

In [24]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [25]:
query = "华为手机拍照怎么样"

print(sales(query))

['华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。', '华为手机的相机功能非常丰富，包括夜景模式、超级微距、AI美颜等多种功能，让您在各种场景下都能拍出好照片。', '华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。', '华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。']


In [26]:
print(sales(query, 0.75))

['华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。', '华为手机的相机功能非常丰富，包括夜景模式、超级微距、AI美颜等多种功能，让您在各种场景下都能拍出好照片。', '华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。', '华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。']


In [27]:
query = "华为手机拍照怎么样"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

score:0.8 ans: ['华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。', '华为手机的相机功能非常丰富，包括夜景模式、超级微距、AI美颜等多种功能，让您在各种场景下都能拍出好照片。', '华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。', '华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。']

score:0.75 ans: ['华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。', '华为手机的相机功能非常丰富，包括夜景模式、超级微距、AI美颜等多种功能，让您在各种场景下都能拍出好照片。', '华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。', '华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。']

score:0.5 ans: ['华为手机的相机采用了领先的技术，拍摄效果非常出色，特别适合拍摄高质量的照片和视频。您可以现场试拍一下，感受一下效果。', '华为手机的相机功能非常丰富，包括夜景模式、超级微距、AI美颜等多种功能，让您在各种场景下都能拍出好照片。', '华为手机的设计非常时尚，外观精美，多种颜色可选，满足您对美的追求，同时手感也非常出色。', '华为手机在性能、拍照、续航等方面都处于领先地位，同时我们还提供全球领先的5G技术，确保您拥有最佳的使用体验。']


#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [28]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5,base_url="https://burn.hair/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [29]:
qa_chain({"query": "你们手机拍照好吗？"})

c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们手机拍照好吗？',
 'result': '很抱歉，但作为一个AI，我没有手机，也无法拍照。如果您想了解特定手机的相机性能，我可以提供一些普遍的信息或者针对特定型号的相机性能进行说明。如果您有任何具体的手机型号或品牌想了解，请告诉我，我会尽力提供相关信息。'}

In [30]:
qa_chain({"query": "你们华为手机是安卓系统吗"})

{'query': '你们华为手机是安卓系统吗', 'result': '华为手机原本搭载的是基于Android深度定制的EMUI系统。'}

In [31]:
print(sales("你们手机是安卓系统吗"))

[]


c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


## 加载 FAISS 向量数据库已有结果

In [32]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(base_url="https://burn.hair/v1"),allow_dangerous_deserialization=True)

In [33]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5,base_url="https://burn.hair/v1")
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [34]:
qa_chain({"query": "我想买别墅，你们有么"})

c:\Users\fengbr\AppData\Local\anaconda3\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [ ]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助你购买别墅，因为我是一个人工智能，没有提供房地产服务的功能。'}

In [ ]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [ ]:
result = qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [ ]:
result

{'query': '我想买别墅，你们有么',
 'result': '对不起，我不能帮你购买别墅。我是一个AI助手，我主要用来提供信息和回答问题。',
 'source_documents': []}